<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>CatBoost</a></span></li><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#Работа-с-BERT" data-toc-modified-id="Работа-с-BERT-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Работа с BERT</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
!pip3 install -q transformers catboost

Script file 'B:\anaconda\Scripts\pip3-script.py' is not present.


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import transformers
import re
from tqdm import tqdm
from pymystem3 import Mystem
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline
from catboost import CatBoostClassifier

In [3]:
df = pd.read_csv('/datasets/toxic_comments.csv') #для яндекса
#df = pd.read_csv('/content/drive/MyDrive/toxic_comments.csv') # для колаба
#df = pd.read_csv('toxic_comments.csv') # для локального

In [4]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [6]:
df['text'].isna().sum()

0

In [7]:
df['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Наблюдается явный дисбаланс классов, поэтому обучать модели будем с использования балансировки весов

Создаем корпус, очищаем его, лемматизируем

In [8]:
!pip install spacy

In [9]:
import spacy

In [10]:
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
def clear_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

In [11]:
df

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \n\nThat is ...,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,159449,And it looks like it was actually you who put ...,0


In [12]:
from tqdm.notebook import tqdm
tqdm.pandas()

df['lemm_text'] = df['text'].progress_apply(clear_text)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [13]:
features = df['lemm_text']
target = df['toxic']

features_train, features_test, target_train_1, target_test_1 = train_test_split(features, target, \
                                                                            test_size=0.2, random_state=12345)

features_train, features_valid, target_train_1, target_valid_1 = train_test_split(features_train, target_train_1, \
                                                                            test_size=0.25, random_state=12345)


In [17]:
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\delo-\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
count_tf_idf = TfidfVectorizer(stop_words=list(stopwords))
features_train_idf = count_tf_idf.fit_transform(features_train)
features_test_idf = count_tf_idf.transform(features_test)
features_valid_idf = count_tf_idf.transform(features_valid)

In [19]:
display(features_train_idf.shape)
display(features_test_idf.shape)
display(features_valid_idf.shape)
display(target_train_1.shape)
display(target_test_1.shape)
display(target_valid_1.shape)

(95574, 129055)

(31859, 129055)

(31859, 129055)

(95574,)

(31859,)

(31859,)

CatBoost постоянно убивал ядро, поэтому я сократил выборку (только для catboost)

In [20]:
features_train_cat = features_train[:6000]
features_test_cat = features_test[:2000]
features_valid_cat = features_valid[:2000]

target_train_cat = target_train_1[:6000]
target_test_cat = target_test_1[:2000]
target_valid_cat = target_valid_1[:2000]

In [21]:
features_train_cat = count_tf_idf.fit_transform(features_train_cat)
features_test_cat = count_tf_idf.transform(features_test_cat)
features_valid_cat = count_tf_idf.transform(features_valid_cat)

In [22]:
display(features_train_cat.shape)
display(features_test_cat.shape)
features_valid_cat.shape

(6000, 23480)

(2000, 23480)

(2000, 23480)

## Обучение

### CatBoost

In [26]:
%%time

cat = CatBoostClassifier(iterations=50)

parameters = { 'depth': range(2, 8),
               'learning_rate': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}

cat_grid = GridSearchCV(cat, parameters, scoring='f1', cv=3)
cat_grid.fit(features_train_cat, target_train_cat, verbose=10)


0:	learn: 0.4125592	total: 158ms	remaining: 7.73s
10:	learn: 0.2287147	total: 367ms	remaining: 1.3s
20:	learn: 0.2044741	total: 586ms	remaining: 809ms
30:	learn: 0.1823484	total: 773ms	remaining: 474ms
40:	learn: 0.1699149	total: 950ms	remaining: 209ms
49:	learn: 0.1583551	total: 1.1s	remaining: 0us
0:	learn: 0.4375281	total: 18.3ms	remaining: 896ms
10:	learn: 0.2300027	total: 193ms	remaining: 684ms
20:	learn: 0.1991806	total: 393ms	remaining: 542ms
30:	learn: 0.1815484	total: 587ms	remaining: 360ms
40:	learn: 0.1690837	total: 784ms	remaining: 172ms
49:	learn: 0.1571250	total: 955ms	remaining: 0us
0:	learn: 0.4367547	total: 18.4ms	remaining: 902ms
10:	learn: 0.2263456	total: 192ms	remaining: 681ms
20:	learn: 0.1979809	total: 371ms	remaining: 512ms
30:	learn: 0.1790905	total: 545ms	remaining: 334ms
40:	learn: 0.1658688	total: 723ms	remaining: 159ms
49:	learn: 0.1573206	total: 916ms	remaining: 0us
0:	learn: 0.3726025	total: 20.5ms	remaining: 1s
10:	learn: 0.2201425	total: 207ms	remaining

GridSearchCV(cv=3,
             estimator=<catboost.core.CatBoostClassifier object at 0x000001D8E23DB0D0>,
             param_grid={'depth': range(2, 8),
                         'learning_rate': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9]},
             scoring='f1')

In [27]:
print(cat_grid.best_params_)
print(cat_grid.best_score_)

{'depth': 7, 'learning_rate': 0.6}
0.6242774463290073


In [28]:
cat = CatBoostClassifier(iterations=50, depth=6, learning_rate=0.7)
cat.fit(features_train_cat, target_train_cat, verbose=10)
print(f1_score(cat.predict(features_valid_cat), target_valid_cat))

0:	learn: 0.3173076	total: 119ms	remaining: 5.84s
10:	learn: 0.1676453	total: 1.2s	remaining: 4.26s
20:	learn: 0.1256960	total: 2.28s	remaining: 3.15s
30:	learn: 0.0994736	total: 3.37s	remaining: 2.06s
40:	learn: 0.0829423	total: 4.47s	remaining: 982ms
49:	learn: 0.0720558	total: 5.45s	remaining: 0us
0.6413043478260869


f1 у catboost слишком мал. Данная модель нам не подходит

### LogisticRegression

In [29]:
%%time
lr_model = LogisticRegression(class_weight='balanced', solver='liblinear', random_state=12345)

parameters = { 'C': [0.1, 1, 10]}

lr_grid = GridSearchCV(lr_model, parameters, scoring='f1', cv=3)
lr_grid.fit(features_train_idf, target_train_1)

CPU times: total: 25 s
Wall time: 6.51 s


GridSearchCV(cv=3,
             estimator=LogisticRegression(class_weight='balanced',
                                          random_state=12345,
                                          solver='liblinear'),
             param_grid={'C': [0.1, 1, 10]}, scoring='f1')

In [30]:
print(lr_grid.best_params_)
print(lr_grid.best_score_)

{'C': 10}
0.7630295788392444


f1 больше необходимого минимума, а значит эту модеель можно тестировать нас тестовой выборке

In [31]:
lr_model = LogisticRegression(class_weight='balanced', solver='liblinear', random_state=12345, C=10)
lr_model.fit(features_train_idf, target_train_1)

LogisticRegression(C=10, class_weight='balanced', random_state=12345,
                   solver='liblinear')

In [32]:
print(f1_score(lr_model.predict(features_valid_idf), target_valid_1))

0.7555168408826947


F1 данной модели больше 0.75, а значит эта модель нам подходит

### Работа с BERT

In [33]:
df

,Unnamed: 0,text,toxic,lemm_text
0,0,Explanation\nWhy the edits made under my usern...,0,Explanation \n why the edit make under my user...
1,1,D'aww! He matches this background colour I'm s...,0,D'aww ! he match this background colour I be s...
2,2,"Hey man, I'm really not trying to edit war. It...",0,"hey man , I be really not try to edit war . it..."
3,3,"""\nMore\nI can't make any real suggestions on ...",0,""" \n More \n I can not make any real suggestio..."
4,4,"You, sir, are my hero. Any chance you remember...",0,"you , sir , be my hero . any chance you rememb..."
...,...,...,...,...
159287,159446,""":::::And for the second time of asking, when ...",0,""" : : : : : and for the second time of asking ..."
159288,159447,You should be ashamed of yourself \n\nThat is ...,0,you should be ashamed of yourself \n\n that be...
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0,"Spitzer \n\n Umm , there s no actual article f..."
159290,159449,And it looks like it was actually you who put ...,0,and it look like it be actually you who put on...


Берем 5000 строк от исходной выборки, иначе эмбендинги мы будем ждать вечность (хотя чаще ядро просто умирает)

In [34]:
df = df.sample(5000).reset_index(drop=True)
features = df['text']
target = df['toxic']

Токенизируем, создаем падинги и маску

In [35]:
tokenizer = transformers.BertTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

tokenized = features.apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=64, truncation=True))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [36]:
model = transformers.BertModel.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")


Some weights of the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Получаем эмбендинги

In [38]:
batch_size = 100
embeddings = []
for i in tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/50 [00:00<?, ?it/s]

разделяем выборку на тестовую и тренировочную

In [39]:
features = np.concatenate(embeddings)
features_train, features_test, target_train, target_test = train_test_split(features, target, \
                                                                            test_size = 0.25, random_state=12345)

features_train, features_valid, target_train, target_valid = train_test_split(features_train, target_train, \
                                                                            test_size=0.25, random_state=12345)

Обучаем LogisticRegression с помощью эмбендингов

In [40]:
bert_lr_model = LogisticRegression(class_weight='balanced', solver='liblinear', random_state=12345, C=10)
bert_lr_model.fit(features_train, target_train)
print(f1_score(bert_lr_model.predict(features_valid), target_valid))


0.5925925925925927


Обучаем CatBoostClassifier с помощью эмбендингов

In [41]:
cat = CatBoostClassifier(iterations=50, depth=6, learning_rate=0.7, random_state=12345)
cat.fit(features_train, target_train, verbose=10)
print(f1_score(cat.predict(features_valid), target_valid))


0:	learn: 0.2695310	total: 101ms	remaining: 4.93s
10:	learn: 0.0901517	total: 948ms	remaining: 3.36s
20:	learn: 0.0398013	total: 1.81s	remaining: 2.49s
30:	learn: 0.0244493	total: 2.68s	remaining: 1.64s
40:	learn: 0.0139126	total: 3.71s	remaining: 814ms
49:	learn: 0.0099474	total: 4.56s	remaining: 0us
0.5609756097560976


Ни одна из моделей (обученных на эмбендингах) не дала нужного нам результата (F1>0.75)

Едиственная модель, которая дала нужный нам скор на валидационной выборке - LogisticRegression. Протестируем ее на тестовой выборке

In [42]:
print(f1_score(lr_model.predict(features_test_idf), target_test_1))

0.772502184678124


F1 > 0.75 -> модель нам подходит

## Выводы

In [49]:
d = {'name' : ['CatBoost', 'LogisticRegression', 'CatBoost_bert', 'LogisticRegression_bert'], \
     'f1_train(cv)' : ['0.624', '0.763', '-', '-'], \
     'f1_valid' : [0.641, 0.755, 0.56, 0.593], \
     'f1_test' : ['-', '0.773', '-', '-']}

result_table = pd.DataFrame(data=d)

Таблица:

In [50]:
display(result_table.sort_values('f1_test', ascending=False))

,name,f1_train(cv),f1_valid,f1_test
1,LogisticRegression,0.763,0.755,0.773
0,CatBoost,0.624,0.641,-
2,CatBoost_bert,-,0.560,-
3,LogisticRegression_bert,-,0.593,-


План выполнения задачи: 
1. Сначала мы выполнили лемматизацию текста, очистили его от стоп слов и создали "корпус".
2. Далее выполни векторизацию с помощью TF-IDF величины 
3. Обучили модели CatBoost и LogisticRegression на векторизованных текстах 
4. Волспользовались BERT моделью. Токенизировали исходные твиты и нашли эмбендингию 
5. На полученных эмбендингах обучили CatBoost и LogisticRegression модели

Единственной моделью у которой F1 > 0.75 является LogisticRegression

Данная модель имеет следующие гиперпараметры:

* class_weight='balanced'
* solver='liblinear'
* C=10

F1 этой модели на тестовой выборке равен: 0.7649074708704593

